In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 10.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 99.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 56.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 6.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# !pip freeze

In [ ]:
import re
import math
import torch
import random
import pandas as pd
from datasets import Dataset
from peft import (
    LoraConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training
)
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    pipeline,
    BitsAndBytesConfig,
)

import warnings
warnings.filterwarnings('ignore')

In [ ]:
peft_lora_model_path = "/content/gdrive/MyDrive/Generative_AI/GPTNEO2.7B/checkpoint-8962"
model_id="EleutherAI/gpt-neo-2.7B"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(model_id, load_in_4bit=True,
                                             torch_dtype = torch.bfloat16,
                                             device_map = {"":0}
                                             )


In [ ]:
model = PeftModel.from_pretrained(model, peft_lora_model_path)

In [ ]:
INTRO = "Below is an instruction that describes a task, paired with an input that provides further context. \nWrite a response that appropriately completes the request."
INSTRUCTION_FORMAT = (
    """{intro}\n\n### Instruction: \n{instruction}\n\n### Input: {input}\n\n ### Response: """
)


In [ ]:
def generate_response(
    instruction: str,
    input_text: str,
    *,
    model,
    tokenizer,
    do_sample: bool = True,
    max_new_tokens: int = 512,
    top_p: float = 0.92,
    top_k: int = 0,
    **kwargs,
) -> str:
    input_ids = tokenizer(
        INSTRUCTION_FORMAT.format(
            intro=INTRO, instruction=instruction, input=input_text
        ),
        return_tensors="pt",
    ).input_ids
    print(input_ids)
    gen_tokens = model.generate(
        input_ids=input_ids,
        pad_token_id=tokenizer.pad_token_id,
        do_sample=do_sample,
        max_new_tokens=max_new_tokens,
        top_p=top_p,
        top_k=top_k,
        **kwargs,
    )
    decoded = tokenizer.batch_decode(gen_tokens)[0]

    # The response appears after "### Response:".  The model has been trained to append "### End" at the end.
    m = re.search(r"#+\s*Response:\s*(.+?)#+\s*End", decoded, flags=re.DOTALL)

    response = None
    if m:
        response = m.group(1).strip()
    else:
        # The model might not generate the "### End" sequence before reaching the max tokens.  In this case, return
        # everything after "### Response:".
        m = re.search(r"#+\s*Response:\s*(.+)", decoded, flags=re.DOTALL)
        if m:
            response = m.group(1).strip()
        else:
            print(f"Failed to find response in:\n{decoded}")

    return response


In [ ]:
print("Welcome to the response generation program!")
while True:
    instruction = "If you are a doctor, please answer the medical questions based on user's query"
    input_text = input("Enter the input text: ")
    response = generate_response(
        instruction=instruction,
        input_text=input_text,
        model=model,
        tokenizer=tokenizer,

        max_new_tokens = 100,
    )
    print('*' * 100)
    print("Generated Response:")
    print(response)
    print('*' * 100)

    continue_generation = input("Do you want to continue (yes/no)? ").lower()
    if continue_generation != "yes":
        print("Exiting the response generation program.")
        break



Welcome to the response generation program!
Enter the input text: i am suffering from common cold and a running nose, what should i do for prevention?
tensor([[21106,   318,   281, 12064,   326,  8477,   257,  4876,    11, 20312,
           351,   281,  5128,   326,  3769,  2252,  4732,    13,   220,   198,
         16594,   257,  2882,   326, 20431, 32543,   262,  2581,    13,   198,
           198, 21017, 46486,    25,   220,   198,  1532,   345,   389,   257,
          6253,    11,  3387,  3280,   262,  3315,  2683,  1912,   319,  2836,
           338, 12405,   198,   198, 21017, 23412,    25,  1312,   716,  7195,
           422,  2219,  4692,   290,   257,  2491,  9686,    11,   644,   815,
          1312,   466,   329, 14196,    30,   628, 44386, 18261,    25,   220]])
****************************************************************************************************
Generated Response:
Yes.  Have cold with running nose are common to occur in a child above 2 years of age. It occu

KeyboardInterrupt: ignored